<a href="https://colab.research.google.com/github/0xtweeter/Advanced-Algorithms/blob/master/Timeseries_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 1.EDA ==> mahdi, amirreza 
- 2.label ==> DeepLOB mahdi
- 3.ml model ==> DNN amirreza|classic ml morteza
- 4.back test ==> morteza 

In [69]:
#download data
import os 
if not os.path.isfile('LOB_NQU22-CME.rar') and not os.path.isfile('LOB_NQU22-CME.csv'):
    !wget https://raw.githubusercontent.com/0xtweeter/Timeseries-Forecasting/main/data/LOB_NQU22-CME.rar
    !pip install unrar
    !unrar x "/content/LOB_NQU22-CME.rar"
    print('data downloaded and exteracted')
    !rm "/content/LOB_NQU22-CME.rar"
elif os.path.isfile('LOB_NQU22-CME.csv'):
    print('csv file already existed.')

--2022-11-10 11:36:14--  https://raw.githubusercontent.com/0xtweeter/Timeseries-Forecasting/main/data/LOB_NQU22-CME.rar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38220604 (36M) [application/octet-stream]
Saving to: ‘LOB_NQU22-CME.rar’

LOB_NQU22-CME.rar   100%[===================>]  36.45M  --.-KB/s    in 0.1s    

2022-11-10 11:36:14 (316 MB/s) - ‘LOB_NQU22-CME.rar’ saved [38220604/38220604]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/LOB_NQU22-CME.rar

Extracting  LOB_NQU22-CME.csv                                             10% 21% 32% 43% 54% 65% 76% 87% 98% 99%

In [70]:
#import libraries
import pandas as pd

# EDA

In [71]:
#reading dataset
pd.set_option('display.max_columns',None)  #to show all columns
colnames=['time', 'open', 'high', 'low', 'close', 'volume', 'numtrades', 'bidvolume', 'askvolume', 'lob_bid', 'lob_ask'] #columns name
data = pd.read_csv("/content/LOB_NQU22-CME.csv", names=colnames, header=None) 

In [72]:
df_eda = data.copy()

In [73]:
df_eda.shape

(72907, 11)

In [74]:
df_eda.head(3)

,time,open,high,low,close,volume,numtrades,bidvolume,askvolume,lob_bid,lob_ask
0,2022-03-18 00:00:00,14066.5,14069.75,14061.75,14066.00,145.0,144.0,60.0,85.0,200.000000|1|578.750000|1|586.500000|1|6490.00...,14019.500000|1|14019.750000|2|14020.000000|1|1...
1,2022-03-18 00:01:00,14064.5,14067.75,14063.00,14066.25,66.0,64.0,36.0,30.0,200.000000|1|578.750000|1|586.500000|1|6490.00...,14020.750000|1|14021.000000|1|14021.250000|1|1...
2,2022-03-18 00:02:00,14067.0,14068.00,14064.75,14066.25,68.0,64.0,36.0,32.0,200.000000|1|578.750000|1|586.500000|1|6490.00...,14022.500000|3|14022.750000|1|14023.000000|1|1...


In [75]:
df_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72907 entries, 0 to 72906
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       72907 non-null  object 
 1   open       72907 non-null  float64
 2   high       72907 non-null  float64
 3   low        72907 non-null  float64
 4   close      72907 non-null  float64
 5   volume     72907 non-null  float64
 6   numtrades  72907 non-null  float64
 7   bidvolume  72907 non-null  float64
 8   askvolume  72907 non-null  float64
 9   lob_bid    72907 non-null  object 
 10  lob_ask    72907 non-null  object 
dtypes: float64(8), object(3)
memory usage: 6.1+ MB


In [62]:
df_eda.describe()

,open,high,low,close,volume,numtrades,bidvolume,askvolume
count,72907.000000,72907.000000,72907.000000,72907.000000,72907.000000,72907.000000,72907.000000,72907.000000
mean,13339.652328,13343.960775,13335.343534,13339.642065,453.720521,413.374491,227.202546,226.517975
std,1264.345092,1263.763152,1264.884332,1264.338144,702.840828,633.438076,354.099034,354.567693
min,11071.000000,11076.750000,11068.500000,11071.250000,1.000000,1.000000,0.000000,0.000000
25%,12123.000000,12127.250000,12118.000000,12122.875000,60.000000,55.000000,29.000000,29.000000
50%,13458.250000,13464.250000,13453.500000,13458.500000,140.000000,128.000000,70.000000,70.000000
75%,14458.250000,14462.500000,14453.750000,14458.250000,600.000000,549.000000,299.000000,298.000000
max,15310.750000,15311.500000,15307.000000,15310.750000,14469.000000,11819.000000,7066.000000,7403.000000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# DATA LABELING

# ML MODELING

## 1.CLASSIC ML

## 2.DNN

# BACK TESTING

# LIVE TRADING